In [33]:
import pandas as pd
import numpy as np

df_pro = pd.read_csv('prolongations.csv', encoding='utf-8')
df_fin = pd.read_csv('financial_data.csv', encoding='utf-8')

# Фильтр 2023 + unique
df_pro['year'] = df_pro['month'].str.extract(r'(\d{4})').astype(int)
df_pro_2023 = df_pro[df_pro['year'] == 2023]
unique_projects = df_pro_2023.groupby('id').agg({'month': 'first', 'AM': 'first'}).reset_index()

#Парсинг отгрузок
month_cols = [col for col in df_fin.columns if '2023' in col]
def parse_amount(val):
    str_val = str(val).strip().replace('\xa0', '').replace(' ', '').replace(',', '.')
    if str_val in ['стоп', 'end']:
        return np.nan
    if str_val == 'в ноль':
        return 0.0
    try:
        return float(str_val)
    except:
        return np.nan

df_fin[month_cols] = df_fin[month_cols].apply(lambda x: x.map(parse_amount))

# Merge + sum дубли
df_fin_grouped = df_fin.groupby('id')[month_cols].sum().reset_index().dropna(how='all')
merged_df = unique_projects.merge(df_fin_grouped, on='id', how='left').fillna(0)
merged_df = merged_df[merged_df['AM'] != 'без А/М']  # Исключить без AM

#Маппинг месяцев
month_map = {'январь 2023': 1, 'февраль 2023': 2, 'март 2023': 3, 'апрель 2023': 4, 'май 2023': 5,
             'июнь 2023': 6, 'июль 2023': 7, 'август 2023': 8, 'сентябрь 2023': 9, 'октябрь 2023': 10,
             'ноябрь 2023': 11, 'декабрь 2023': 12}
target_months_cap = ['Январь 2023', 'Февраль 2023', 'Март 2023', 'Апрель 2023', 'Май 2023',
                     'Июнь 2023', 'Июль 2023', 'Август 2023', 'Сентябрь 2023', 'Октябрь 2023',
                     'Ноябрь 2023', 'Декабрь 2023']

#Единый цикл для баз и коэф
def get_prev(target, steps=1):
    num = month_map[target]
    prev_num = num - steps
    if prev_num < 1:
        return None
    return [m for m, n in month_map.items() if n == prev_num][0]

results = []
for target in month_map.keys():
    prev1 = get_prev(target)
    prev2 = get_prev(target, 2)
    if not prev1:
        continue
    col_target = target_months_cap[month_map[target] - 1]
    col_prev1 = target_months_cap[month_map[prev1] - 1]
    
    prev1_proj = merged_df[merged_df['month'] == prev1]
    base1_total = prev1_proj[col_prev1].sum()
    prolong1_total = prev1_proj[prev1_proj[col_target] > 0][col_target].sum()
    коэф1_total = prolong1_total / base1_total if base1_total > 0 else np.nan
    коэф1_am = prev1_proj.groupby('AM').apply(lambda g: g[g[col_target] > 0][col_target].sum() / g[col_prev1].sum() if g[col_prev1].sum() > 0 else 0)
    
    if prev2:
        col_prev2 = target_months_cap[month_map[prev2] - 1]
        prev2_proj = merged_df[merged_df['month'] == prev2]
        no_prolong1 = prev2_proj[prev2_proj[col_prev1] == 0]
        base2_total = no_prolong1[col_prev2].sum()
        prolong2_total = no_prolong1[no_prolong1[col_target] > 0][col_target].sum()
        коэф2_total = prolong2_total / base2_total if base2_total > 0 else np.nan
        коэф2_am = no_prolong1.groupby('AM').apply(lambda g: g[g[col_target] > 0][col_target].sum() / g[col_prev2].sum() if g[col_prev2].sum() > 0 else 0)
    else:
        base2_total = коэф2_total = np.nan
        коэф2_am = pd.Series()
    
    row = {'target_month': target, 'коэф1_total': коэф1_total, 'коэф2_total': коэф2_total,
           'base1_total': base1_total, 'base2_total': base2_total}
    row.update({'коэф1_' + am: val for am, val in коэф1_am.items()})
    row.update({'коэф2_' + am: val for am, val in коэф2_am.items()})
    results.append(row)

coeffs_df = pd.DataFrame(results)

#Годовой взвешенный
total_base1 = coeffs_df['base1_total'].sum()
coeffs_df['коэф1_year'] = (coeffs_df['коэф1_total'] * coeffs_df['base1_total']).sum() / total_base1 if total_base1 > 0 else np.nan
total_base2 = coeffs_df['base2_total'].sum()
coeffs_df['коэф2_year'] = (coeffs_df['коэф2_total'] * coeffs_df['base2_total']).sum() / total_base2 if total_base2 > 0 else np.nan

#Доп. метрики (кол-во, отгрузка, % изменений, стопы)
projects_df = merged_df.groupby(['month', 'AM']).size().unstack(fill_value=0)
prolong_df = merged_df[merged_df[target_months_cap].gt(0, axis=1)].groupby(['month', 'AM']).size().unstack(fill_value=0)
shipment_df = merged_df.groupby('AM')[target_months_cap].sum()
coeffs_df['%_изменения_коэф1'] = coeffs_df['коэф1_total'].pct_change() * 100
stop_summary = df_fin[df_fin[month_cols].apply(lambda row: row.isna().sum() > 0, axis=1)].groupby('Account').size().reset_index(name='стоп-проекты')
#Сохранение
coeffs_df.to_csv('coeffs_simplified.csv', encoding='utf-8-sig', index=False)
print("Готово: coeffs_simplified.csv")

Готово: coeffs_simplified.csv


C:\Users\shall\AppData\Local\Temp\ipykernel_17204\4135609183.py:61: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  коэф1_am = prev1_proj.groupby('AM').apply(lambda g: g[g[col_target] > 0][col_target].sum() / g[col_prev1].sum() if g[col_prev1].sum() > 0 else 0)
C:\Users\shall\AppData\Local\Temp\ipykernel_17204\4135609183.py:61: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  коэф1_am = prev1_proj.groupby('AM').apply(lamb